# **CAR PRICE PREDICTION - Notebook D** 

**Tópicos neste notebook:**

10 - Deploy do modelo

# **10.0 Deploy Model to Production**

## 10.1 Cars Price Pred Class

In [14]:
import pickle
import pandas as pd
import numpy as np
import math


class CarsPricePred(object):
    def __init__(self):
        self.home_path='C:/Users/andre/repos/cursos_ds/9_car_price_prediction/' # Para teste local da API
        self.ano_de_fabricacao_scaler = pickle.load(open(self.home_path + 'parameter/ano_de_fabricacao.pkl', 'rb'))
        self.ano_modelo_scaler =        pickle.load(open(self.home_path + 'parameter/ano_modelo.pkl', 'rb'))
        self.cilindradas_scaler =       pickle.load(open(self.home_path + 'parameter/cilindradas.pkl', 'rb'))
        self.hodometro_scaler =         pickle.load(open(self.home_path + 'parameter/hodometro.pkl', 'rb'))
        self.cambio_encoder =           pickle.load(open(self.home_path + 'parameter/cambio_tgt_encoder.pkl', 'rb'))
        self.combustivel_encoder =      pickle.load(open(self.home_path + 'parameter/combustivel_tgt_encoder.pkl', 'rb'))
        self.cor_encoder =              pickle.load(open(self.home_path + 'parameter/cor_tgt_encoder.pkl', 'rb'))
        self.modelo_encoder =           pickle.load(open(self.home_path + 'parameter/modelo_tgt_encoder.pkl', 'rb'))
        self.marca_encoder =            pickle.load(open(self.home_path + 'parameter/marca_tgt_encoder.pkl', 'rb'))
        self.tipo_encoder =             pickle.load(open(self.home_path + 'parameter/tipo_tgt_encoder.pkl', 'rb'))

    # -----------------------------------------------
    # Observação!!!
    # Nesse caso, não será necessário tratar os dados, pois as informações que serão enviadas do streamlit já estão tratadas
    # def data_cleaning( self, df1 ):
    #     return df1
    # def feature_engineering( self, df2):
    #     return df2
    # -----------------------------------------------
        

    def data_preparation(self, df):

        ## Rescaling and Encoding
        self.ano_de_fabricacao_scaler.transform(df[['ano_de_fabricacao']].values)
        self.ano_modelo_scaler.transform(df[['ano_modelo']].values)
        self.cilindradas_scaler.transform(df[['cilindradas']].values)
        self.hodometro_scaler.transform(df[['hodometro']].values)
        self.cambio_encoder.transform(df[['cambio']])
        self.combustivel_encoder.transform(df[['combustivel']])
        self.cor_encoder.transform(df[['cor']])
        self.modelo_encoder.transform(df[['modelo']])
        self.marca_encoder.transform(df[['marca']])
        self.tipo_encoder.transform(df[['tipo']])
        df['blindado'] = df['blindado'].apply(lambda x: 1 if x == 'Sim' else 0)

        return df    

    def get_prediction(self, model, df_car, df_to_pred):

        # prediction
        price_pred = model.predict(df_to_pred)

        # join pred into the original data
        df_car['preco_sugerido'] = np.expm1(price_pred)

        return df_car.to_json(orient='records')

## 10.2 API Handler 

In [4]:
import os
import pickle
import pandas as pd
from flask import Flask, request, Response
from api.cars_pred.CarsPricePred import CarsPricePred

# Loading ML model
model = pickle.load(open('C:/Users/andre/repos/cursos_ds/9_car_price_prediction/model/model_car_price_pred.pkl', 'rb'))

# Initialize API
app = Flask(__name__)
@app.route('/api-cars-price-predict', methods=['POST'])

def rossmann_predict():
    car_json = request.get_json()

    if car_json: # there is data
        if isinstance(car_json, dict): # unique example
            df_car = pd.DataFrame(car_json, index=[0])
        else: # multiple example
            df_car = pd.DataFrame(car_json, columns=car_json[0].keys())
            #[0].keys() para usar como colunas as primeiras chaves no df 

        # Instantiate CarsPricePred class
        pipeline = CarsPricePred()

        # -----------------------------------------------
        # Observação!!!
        # Não necessário nesse caso.
        # data cleaning
        # df1 = pipeline.data_cleaning( df_car )
        # feature engineering
        # df2 = pipeline.feature_engineering( df1 )
        # -----------------------------------------------

        # data preparation
        df_to_pred = pipeline.data_preparation(df_car)

        # prediction
        df_response = pipeline.get_prediction(model, df_car, df_to_pred)

        return df_response

    else:
        return Response('{}', status=200, mimetype='application/json')

if __name__ == '__main__':
    app.run( '0.0.0.0' )

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.8:5000
Press CTRL+C to quit


## 10.3 Testing

In [15]:
import json
import requests

df5 = pd.read_csv('datasets/cooked_datasets/df3_processed.csv', low_memory = False)

In [16]:
# Separando uma linha como amostra para teste
features_selecionadas = ['marca', 'cor', 'combustivel', 'tipo', 'cilindradas', 'cidade_vendedor','ano_de_fabricacao', 'ano_modelo', 'hodometro', 'modelo', 'cambio', 'blindado']
linha_selecionada = df5[features_selecionadas].iloc[0]

# Converte a linha para um dicionário
linha_dict = linha_selecionada.to_dict()
linha_json = json.dumps(linha_dict)
print(linha_json)

{"marca": "FORD", "cor": "Branco", "combustivel": "DIESEL", "tipo": "Picape", "cilindradas": 3.2, "cidade_vendedor": "Itapiranga", "ano_de_fabricacao": 2017, "ano_modelo": 2018, "hodometro": 51157.0, "modelo": "RANGER", "cambio": "Automatica", "blindado": "Nao"}


In [20]:
# API Call

# Teste local
url = 'http://127.0.0.1:5000/api-cars-price-predict'

# Teste Web Service
# url = 'https://api-cars-price-predict.onrender.com/api-cars-price-predict'

header = {'Content-type': 'application/json' }
data = linha_json

r = requests.post(url, data=linha_json, headers=header)

print(f'Status Code {r.status_code}')

# Manter o response_json recebendo r.json()
response_json = r.json()
preco_sugerido = response_json[0].get('preco_sugerido')
if preco_sugerido is not None:
    print(preco_sugerido)
else:
    print("Chave 'preco_sugerido' não encontrada no JSON.")

Status Code 200
207212.046875
